In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np

#初始化模型和tokenizer
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [38]:
def get_embedding(text, is_query=False):
    #增加前綴
    if is_query:
        text=f'query:{text}'
    else:
        text=f'passage:{text}'

    #編碼文字內容
    inputs = tokenizer(text,padding=True, truncation=True, max_length=512, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    

    #獲取嵌入向量
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :] # 使用 [CLS] token的输出
        embeddings = torch.nn.functional.normalize(embeddings,p=2, dim=1)

    return embeddings.cpu().numpy()[0]
        

def create_document_embeddings(df):
    """為document建立嵌入向量"""
    embeddings = []
    for _, row in df.iterrows(): #row是Series
        text = f"Title: {row['Title']}\nText: {row['Text']}"
        embedding = get_embedding(text, is_query=False)
        embeddings.append(embedding)
    return embeddings

def find_best_passage(query, df):
    '''搜尋相關的內容'''
    #獲取查詢的嵌入向量
    query_embedding = get_embedding(query, is_query=True)

    #計算相似度
    similarities = np.dot(np.stack(df['Embeddings'].values),query_embedding) 
    
    #傳品最相關的本文
    best_idx = np.argmax(similarities)

    return df.iloc[best_idx]['Text'], similarities[best_idx]


In [40]:
# 准备示例数据
documents = [
    {
        "Title": "操作氣候控制系統",
        "Text": "您的 Googlecar 配備氣候控制系統，可讓您調節車內的溫度和氣流。若要操作氣候控制系統，請使用中央控制台上的按鈕和旋鈕。"
    },
    {
        "Title": "觸控螢幕",
        "Text": "您的 Googlecar 擁有大型觸控螢幕顯示屏，可使用各種功能，包括導航、娛樂和氣候控制。"
    }
]
#建立DataFrame
df = pd.DataFrame(documents)
df['Embeddings'] = create_document_embeddings(df)
query = "有導航,娛樂和氣候"
best_text, similarity = find_best_passage(query, df)
print(f"Query:{query}")
print(f'best matching text(similarity:{similarity:.3f})')
print(best_text)



Query:有導航,娛樂和氣候
best matching text(similarity:0.876)
您的 Googlecar 擁有大型觸控螢幕顯示屏，可使用各種功能，包括導航、娛樂和氣候控制。
